In [13]:
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.sentiment.vader import SentimentIntensityAnalyzer

import datetime

# Download NLTK resources
nltk.download('punkt_tab')
nltk.download('stopwords')

# Download the VADER lexicon
nltk.download('vader_lexicon')



[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\idcsa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\idcsa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\idcsa\AppData\Roaming\nltk_data...


True

In [5]:
# Initialize the books DataFrame
books_df = pd.DataFrame(columns=["Title", "Author", "Genre", "Availability"])

# Function to add a new book
def add_book(title, author, genre):
    global books_df
    new_book = pd.DataFrame({"Title": [title], "Author": [author], "Genre": [genre], "Availability": ["Available"]})
    books_df = pd.concat([books_df, new_book], ignore_index=True)
    print(f"Book '{title}' by {author} added successfully!")

# Function to remove a book
def remove_book(title):
    global books_df
    books_df = books_df[books_df['Title'] != title]
    print(f"Book '{title}' removed successfully!")

# Add sample books
add_book("The Great Gatsby", "F. Scott Fitzgerald", "Fiction")
add_book("1984", "George Orwell", "Dystopian")
add_book("To Kill a Mockingbird", "Harper Lee", "Fiction")

# Display the books
books_df


Book 'The Great Gatsby' by F. Scott Fitzgerald added successfully!
Book '1984' by George Orwell added successfully!
Book 'To Kill a Mockingbird' by Harper Lee added successfully!


,Title,Author,Genre,Availability
0,The Great Gatsby,F. Scott Fitzgerald,Fiction,Available
1,1984,George Orwell,Dystopian,Available
2,To Kill a Mockingbird,Harper Lee,Fiction,Available


In [25]:
# Function to search for books
def search_books(query):
    global books_df
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(query.lower())
    filtered_words = [word for word in words if word.isalnum() and word not in stop_words]
    
    results = books_df[
        books_df['Title'].str.lower().str.contains('|'.join(filtered_words)) |
        books_df['Author'].str.lower().str.contains('|'.join(filtered_words)) |
        books_df['Genre'].str.lower().str.contains('|'.join(filtered_words))
    ]
    
    if not results.empty:
        print(f"Books matching '{query}':")
        print(results)
    else:
        print(f"No books found for query: '{query}'")

# Example search
search_books("Ben 10")


No books found for query: 'Ben 10'


In [16]:
# Initialize the VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Function to analyze sentiment of a review
def analyze_review_sentiment(review):
    sentiment_scores = sid.polarity_scores(review)
    print(f"Review Sentiment Analysis: {sentiment_scores}")
    if sentiment_scores['compound'] >= 0.05:
        print("Overall Sentiment: Positive")
    elif sentiment_scores['compound'] <= -0.05:
        print("Overall Sentiment: Negative")
    else:
        print("Overall Sentiment: Neutral")

# Example review analysis
analyze_review_sentiment("This is nicer than expected but it could be better!")


Review Sentiment Analysis: {'neg': 0.0, 'neu': 0.568, 'pos': 0.432, 'compound': 0.7263}
Overall Sentiment: Positive


In [21]:
# Initialize the borrowers DataFrame
borrowers_df = pd.DataFrame(columns=["Borrower Name", "Book Title", "Borrow Date", "Due Date"])

# Function to add a borrower
def add_borrower(borrower_name, book_title):
    global borrowers_df, books_df
    if books_df.loc[books_df['Title'] == book_title, 'Availability'].values[0] == "Available":
        borrow_date = datetime.date.today()
        due_date = borrow_date + datetime.timedelta(days=14)  # 2 weeks borrowing period
        new_borrower = pd.DataFrame({
            "Borrower Name": [borrower_name], 
            "Book Title": [book_title], 
            "Borrow Date": [borrow_date], 
            "Due Date": [due_date]
        })
        borrowers_df = pd.concat([borrowers_df, new_borrower], ignore_index=True)
        # Update book availability
        books_df.loc[books_df['Title'] == book_title, 'Availability'] = "Borrowed"
        print(f"'{book_title}' has been borrowed by {borrower_name}.")
    else:
        print(f"Sorry, '{book_title}' is currently not available.")

# Example of adding a borrower
add_borrower("Alice Johnson", "1984")

# Display borrowers
borrowers_df


'1984' has been borrowed by Alice Johnson.


,Borrower Name,Book Title,Borrow Date,Due Date
0,Alice Johnson,1984,2024-09-05,2024-09-19


In [23]:
# Function to return a book
def return_book(borrower_name, book_title):
    global borrowers_df, books_df
    borrowers_df = borrowers_df[
        ~(borrowers_df['Borrower Name'] == borrower_name) & (borrowers_df['Book Title'] == book_title)
    ]
    books_df.loc[books_df['Title'] == book_title, 'Availability'] = "Available"
    print(f"'{book_title}' has been returned by {borrower_name} and is now available.")

# Example of returning a book
return_book("Alice Johnson", "1984")

# Display updated borrowers and books
borrowers_df, books_df


'1984' has been returned by Alice Johnson and is now available.


(Empty DataFrame
 Columns: [Borrower Name, Book Title, Borrow Date, Due Date]
 Index: [],
                    Title               Author      Genre Availability
 0       The Great Gatsby  F. Scott Fitzgerald    Fiction    Available
 1                   1984        George Orwell  Dystopian    Available
 2  To Kill a Mockingbird           Harper Lee    Fiction    Available)

In [26]:
add_book("Ben 10", "", "Fiction")


Book 'Ben 10' by  added successfully!


In [28]:
search_books("Ben Ten")

Books matching 'Ben Ten':
    Title Author    Genre Availability
3  Ben 10         Fiction    Available


In [27]:
books_df

,Title,Author,Genre,Availability
0,The Great Gatsby,F. Scott Fitzgerald,Fiction,Available
1,1984,George Orwell,Dystopian,Available
2,To Kill a Mockingbird,Harper Lee,Fiction,Available
3,Ben 10,,Fiction,Available
